In [1]:
import torch
from utils import get_model_and_tokenizer
model, tokenizer = get_model_and_tokenizer()

/opt/homebrew/Caskroom/miniconda/base/envs/311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Generate random pages
import random
from utils import topk_gradient
from data import SubsetFalconLoader
pages = [random.randint(0, SubsetFalconLoader.max_pages) for _ in range(1)]
        
# Create batches of data to process using the SubsetFalconLoader with the given parameters
batches = list(
    SubsetFalconLoader(
        tokenizer = tokenizer,
        batch_size = 1, 
        sequence_length = 50,
        rows = pages
    )
)

# Move the model to the specified device (CPU or GPU)
model.to('cpu')

# Reset gradients in the model to zero
model.zero_grad()

# Process each batch of data
for batch in batches:
    # Move the batch to the specified device
    inputs = batch.to('cpu')
    # Pass the inputs through the model and calculate the loss
    outputs = model(inputs, labels=inputs)
    # Normalize the loss by the number of batches
    outputs.loss /= len(batches)
    # Backpropagate the loss to compute gradients
    outputs.loss.backward()
    # Exit the loop after processing the first batch for demonstration purposes
    break

# Extract the top-k percent gradients from the model
gradient = topk_gradient( model, 0.01 )

In [4]:
import torch
import boto3
import tempfile
import os

import time

import paramiko
import os

# EC2 instance details
instance_name = 'i-0b3994e1d8b4042f2'
ec2_hostname = 'ec2-3-231-208-20.compute-1.amazonaws.com'  # Placeholder for actual hostname
ec2_username = 'ec2-user'
private_key_path = os.path.expanduser('~/.ssh/main.pem')  # Placeholder for actual path to your private key

# Save the gradient to a temporary file and automatically delete it after use
with tempfile.NamedTemporaryFile() as tmp_file:
    torch.save(gradient, tmp_file.name)

    # Measure the time it takes to commit the file to the EC2 instance
    start_time = time.time()

    # Establish SSH connection to the EC2 instance
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    ssh.connect(ec2_hostname, username=ec2_username, key_filename=private_key_path)

    # SCP the temporary file to the EC2 instance
    scp = paramiko.SFTPClient.from_transport(ssh.get_transport())
    remote_path = f'/home/{ec2_username}/gradient_file.pt'  # Placeholder for actual remote path
    scp.put(tmp_file.name, remote_path)
    scp.close()
    ssh.close()

    # Calculate and print the commit duration
    commit_duration = time.time() - start_time
    print(f"File committed to EC2 instance {instance_name} in {commit_duration} seconds")


File committed to EC2 instance i-0b3994e1d8b4042f2 in 23.90545916557312 seconds
